<a href="https://colab.research.google.com/github/kunalsonalkar/training-llms/blob/main/fineTuning_gpt2_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rasbt/LLMs-from-scratch.git

Cloning into 'LLMs-from-scratch'...
remote: Enumerating objects: 5555, done.
remote: Total 5555 (delta 0), reused 0 (delta 0), pack-reused 5555 (from 1)
Receiving objects: 100% (5555/5555), 12.92 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (3479/3479), done.


In [2]:
import pandas as pd

try:
    labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='latin1',usecols=[0,1])
except UnicodeDecodeError:
    labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='cp1252', usecols=[0,1])
# If none of the encoding above work, you may also try:
# labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='utf-16')
# labels = pd.read_csv('/content/LLMs-from-scratch/ch06/01_main-chapter-code/query_intent_labels.csv', encoding='iso-8859-1')

In [3]:
import pandas as pd

# Your category-to-integer mapping
category_map = {
    'baby_accessories_a28': 0,
    'bags_a17': 1,
    'belts__braces_a26': 2,
    'bottoms_a3': 3,
    'dresses_a1': 4,
    'eyewear_a19': 5,
    'food_a38': 6,
    'fragrance_a31': 7,
    'gloves_mittens_a21': 8,
    'hair_accessories_a20': 9,
    'hair_care_a33': 10,
    'headwear_a18': 11,
    'home_a35': 12,
    'hosiery_a16': 13,
    'jacket_sportcoat_mg12': 14,
    'jewelry_a14': 15,
    'jewelry_care_a15': 16,
    'jumpsuits_coveralls_a4': 17,
    'makeup_a30': 18,
    'neckwear_a27': 19,
    'outerwear_a11': 20,
    'personal_care_accessories_a34': 21,
    'scarves_wraps_ponchos_a22': 22,
    'shoe_care_a13': 23,
    'shoes_a12': 24,
    'skin_body_treatment_a32': 25,
    'sleepwear_a9': 26,
    'small_leather_goods_a23': 27,
    'stationery_giftwrap_a36': 28,
    'suits_sets_wardrobers_mg5': 29,
    'swimwear_a8': 30,
    'toys_games_a37': 31,
    'tops_a2': 32,
    'underwear_lingerie_a7': 33
}


# Map categories to integers
labels['category_int'] = labels['product_type_1_label'].map(category_map)
print(labels)


              search_term     product_type_1_label  category_int
0             womens uggs                shoes_a12            24
1       charlotte tilbury               makeup_a30            18
2     womens birkenstocks                shoes_a12            24
3                    uggs                shoes_a12            24
4            womens nikes                shoes_a12            24
...                   ...                      ...           ...
3684        girls pajamas             sleepwear_a9            26
3685          hand towels                 home_a35            12
3686            hand wash  skin_body_treatment_a32            25
3687       l'agence jeans               bottoms_a3             3
3688             lipgloss               makeup_a30            18

[3689 rows x 3 columns]


In [4]:
def random_split(df, train_frac=0.7, validation_frac=0.2):
  df = df.sample(frac=1, random_state=42).reset_index(drop=True)
  train_end = int(len(df) * train_frac)
  validation_end = train_end + int(len(df) * validation_frac)
  train_df = df[:train_end]
  validation_df = df[train_end:validation_end]
  test_df = df[validation_end:]
  return train_df, validation_df, test_df

In [5]:
train_df, validation_df, test_df = random_split(labels, train_frac=0.7, validation_frac=0.1)

In [6]:
train_df.to_csv('train.csv', index=None)
validation_df.to_csv('validation.csv', index=None)
test_df.to_csv('test.csv', index=None)

In [7]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00


In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.decode([50256]))

<|endoftext|>


In [9]:
import torch
from torch.utils.data import Dataset

class queryIntent(Dataset):
  def __init__(self, csv_file, tokenizer, max_length = None, pad_token_id=50256):
      self.data = pd.read_csv(csv_file)
      self.encoded_texts = [
          tokenizer.encode(text) for text in self.data["search_term"]
      ]
      if max_length is None:
        self.max_length = self._longest_encoded_length()
      else:
        self.max_length = max_length
        self.encoded_texts = [
            encoded_text[:self.max_length] for encoded_text in self.encoded_texts
        ]

      self.encoded_texts = [
          encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
          for encoded_text in self.encoded_texts
      ]

  def __getitem__(self, index):
      encoded = self.encoded_texts[index]
      label = self.data.iloc[index]["category_int"]
      return (
          torch.tensor(encoded, dtype=torch.long),
          torch.tensor(label, dtype=torch.long)
      )

  def __len__(self):
      return len(self.data)

  def _longest_encoded_length(self):
      max_length = 0
      for encoded_text in self.encoded_texts:
          encoded_length = len(encoded_text)
          if encoded_length > max_length:
              max_length = encoded_length
      return max_length

In [10]:
train_dataset = queryIntent("train.csv", max_length=None, tokenizer=tokenizer)
validation_dataset = queryIntent("validation.csv", max_length = None, tokenizer=tokenizer)
test_dataset = queryIntent("test.csv", max_length = None, tokenizer=tokenizer)

In [11]:
from torch.utils.data import DataLoader
num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True
)

validation_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

In [12]:
len(train_loader)

322

In [13]:
len(test_loader)

93

In [14]:
len(validation_loader)

46

In [15]:
len(train_dataset)

2582

In [16]:
for input_batch, label_batch in train_loader:
    print(input_batch.shape)
    print(label_batch.shape)
    break

torch.Size([8, 11])
torch.Size([8])


**Initialize the model and Pretrained Weights**

In [17]:
CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True,
    "emb_dim": 1024,
    "n_layers": 24,
    "n_heads": 16,
    "n_embd": 1024
}

model_configs = {
    "gpt2-small (124M)": {
        "n_layer": 12,
        "n_head": 12,
        "n_embd": 768
    },
    "gpt2-medium (355M)": {
        "n_layer": 24,
        "n_head": 16,
        "n_embd": 1024
    },
    "gpt2-large (774M)": {
        "n_layer": 36,
        "n_head": 20,
        "n_embd": 1280
    },
    "gpt2-xl (1.5B)": {
        "n_layer": 48,
        "n_head": 25,
        "n_embd": 1600
    }
}

In [18]:
%cd /content/LLMs-from-scratch/ch06/01_main-chapter-code

/content/LLMs-from-scratch/ch06/01_main-chapter-code


In [19]:
import gpt_download
from gpt_download import download_and_load_gpt2

In [20]:
from previous_chapters import GPTModel, load_weights_into_gpt

In [21]:
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir = "gpt2")

model = GPTModel(BASE_CONFIG)


checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 102kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.05MiB/s]
hparams.json: 100%|██████████| 91.0/91.0 [00:00<00:00, 152kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 1.42G/1.42G [04:32<00:00, 5.21MiB/s]
model.ckpt.index: 100%|██████████| 10.4k/10.4k [00:00<00:00, 10.0MiB/s]
model.ckpt.meta: 100%|██████████| 927k/927k [00:00<00:00, 932kiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 609kiB/s]


In [22]:
load_weights_into_gpt(model, params)

In [23]:
from previous_chapters import (
    generate_text_simple,
    text_to_token_ids,
    token_ids_to_text
)

text1 = "Every effort moves you"

token_ids = generate_text_simple(
    model = model,
    idx = text_to_token_ids(text1, tokenizer),
    max_new_tokens = 15,
    context_size = BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Every effort moves you forward, but you must be careful. You must not let your guard down


In [24]:
for param in model.parameters():
  param.requires_grad=False

In [25]:
model.out_head = torch.nn.Linear(BASE_CONFIG["emb_dim"], 34)

In [26]:
model

GPTModel(
  (tok_emb): Embedding(50257, 1024)
  (pos_emb): Embedding(1024, 1024)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=1024, out_features=1024, bias=True)
        (W_key): Linear(in_features=1024, out_features=1024, bias=True)
        (W_value): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU()
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_f

In [27]:
for param in model.trf_blocks[-1].parameters():
  param.requires_grad=True

for param in model.final_norm.parameters():
  param.requires_grad=True

In [28]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions", inputs.shape)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions torch.Size([1, 4])


In [29]:
with torch.no_grad():
  outputs = model(inputs)
print(outputs)

tensor([[[ -1.1405,  -5.2398,   0.4588,  -4.5194,  -2.1759,  -1.6721,   5.6295,
           -3.2461,   3.8272,   2.2588,  -5.4441,  -0.4118,   3.7665,  -4.1872,
            6.3213,  -4.1783,   1.1475,   3.2574,   5.2512,   0.3507,  -5.6813,
            4.7116,  -3.4804,  -5.7551,  -2.2037,  -4.8782,   4.9031,   5.1971,
           -3.1988,  -0.8840,   3.6855,  -0.0292,   3.7163,   1.8945],
         [ -1.7277,  -7.8605,   0.1868,  -9.5683,  -1.2810,   0.3875,   9.0098,
           -6.5969,   7.1907,   4.8082, -11.9986,  -0.1371,   5.5833,  -8.4638,
            9.1116,  -4.4983,   2.2615,   2.2679,   8.7837,  -2.2801, -12.3155,
           10.0300,  -8.0281,  -6.2372,  -3.5183,  -9.2350,  10.4550,  10.2848,
           -7.7542,  -0.6904,   5.3863,  -2.2003,   3.2774,   1.9511],
         [ -0.9232,  -4.6514,  -1.1046,  -6.5513,  -0.6169,   1.2484,   4.8546,
           -5.0220,   4.9890,   3.3734,  -7.4748,  -0.5737,   3.4910,  -6.2440,
            5.1007,  -1.8495,   0.9752,   0.7582,   5.5598

In [30]:
logits = outputs[:,-1,:]

In [31]:
probas = torch.softmax(logits, dim=-1)

In [32]:
probas

tensor([[1.9017e-03, 1.4145e-04, 3.2588e-03, 1.8045e-05, 9.7367e-04, 2.0886e-03,
         6.8988e-02, 2.3598e-05, 6.5930e-02, 2.6094e-02, 1.5802e-05, 1.7519e-03,
         1.5203e-02, 3.7920e-05, 1.0382e-01, 3.0529e-04, 5.0321e-03, 7.3116e-03,
         6.0512e-02, 4.9100e-04, 2.0872e-05, 1.0140e-01, 3.7979e-05, 1.9817e-04,
         7.0630e-04, 5.7304e-05, 1.4905e-01, 3.5391e-01, 1.4480e-04, 6.3619e-04,
         1.0946e-02, 3.3534e-04, 1.3819e-02, 4.8342e-03]])

In [33]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model = model.to(device)
    model.eval()
    correct_predictions, num_examples = 0, 0

    total_batches = len(data_loader) if num_batches is None else min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i >= total_batches:
            break
        # Move data to device
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)
        with torch.no_grad():
            outputs = model(input_batch)[:, -1, :]
        predicted_labels = torch.argmax(outputs, dim=-1)
        num_examples += predicted_labels.shape[0]
        correct_predictions += (predicted_labels == target_batch).sum().item()
    return correct_predictions / num_examples


In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.manual_seed(123)

In [35]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)

In [36]:
train_accuracy

0.02251552795031056

In [37]:
validation_accuracy = calc_accuracy_loader(validation_loader, model, device)
validation_accuracy

0.016304347826086956

In [38]:
def calc_loss_batch(input_batch, target_batch, model, device):
  input_batch, target_batch = input_batch.to(device), target_batch.to(device)
  logits = model(input_batch)[:,-1,:]
  loss = torch.nn.functional.cross_entropy(logits, target_batch)
  return loss

In [39]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
  total_loss = 0
  if len(data_loader) == 0:
    return float("nan")
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))
  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break
  return total_loss/num_batches

In [40]:
train_loss = calc_loss_loader(train_loader, model, device)
train_loss

6.09723520871275

**Fine tuning the model**

In [41]:
def train_classifier_simple(model, train_loader, validation_loader,
                            optimizer, device, num_epochs, eval_freq, eval_iter):
  train_losses, val_losses, train_accs, val_accs = [], [], [], []
  examples_seen, global_step = 0, -1

  for epoch in range(num_epochs):
    model.train()

    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      loss.backward()
      optimizer.step()
      examples_seen += input_batch.shape[0]
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(model, train_loader, validation_loader, device, eval_iter)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Ep {epoch+1} (Step {global_step:06d}): "
              f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
      train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
      train_accs.append(train_accuracy)
      val_accuracy = calc_accuracy_loader(validation_loader, model, device, num_batches=eval_iter)
      val_accs.append(val_accuracy)
      print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
      print(f"Validation accuracy: {val_accuracy*100:.2f}%")
  return train_losses, val_losses, train_accs, val_accs, examples_seen

In [42]:
def evaluate_model(model, train_loader, validation_loader, device, eval_iter):
  model.eval()
  with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
    val_loss = calc_loss_loader(validation_loader, model, device, num_batches=eval_iter)
  model.train()
  return train_loss, val_loss

In [43]:
import time
start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model = model,
    train_loader = train_loader,
    validation_loader = validation_loader,
    optimizer = optimizer,
    device = device,
    num_epochs = num_epochs,
    eval_freq = 50,
    eval_iter = 5
)

end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

Ep 1 (Step 000000): Train loss 6.364, Val loss 6.214
Training accuracy: 0.00% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 5.00% | Validation accuracy: 0.00%
Training accuracy: 5.00% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 0.00% | Validation accuracy: 0.00%
Training accuracy: 0.00% | Validation accuracy: 0.00%
Training accuracy: 0.00% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 5.00% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 7.50% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 2.50% | Validation accuracy: 0.00%
Training accuracy: 0.00% | Validation accuracy: 0.00%
Training accuracy: 0.00% | Va

In [44]:
 device

device(type='cuda')

In [45]:
import torch.nn.functional as F

def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.weight.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()
    probabilities = F.softmax(logits, dim=-1).squeeze(0)

    probabilities = probabilities.cpu().numpy()

    # Return the classified result
    return predicted_label, probabilities

In [58]:
# Example call:
result = classify_review(
    text="nest candle",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=train_dataset.max_length,
    pad_token_id=50256
)
print(result)  # e.g., 'shoes_a12'


(12, array([7.6304867e-05, 2.3086350e-06, 3.3676208e-08, 1.5312900e-09,
       2.5658523e-05, 1.2444661e-05, 2.9612986e-05, 1.5105647e-05,
       5.9128840e-08, 3.7891627e-05, 1.0830632e-05, 7.3475117e-06,
       9.9727243e-01, 4.5772762e-07, 3.6077117e-07, 3.5043812e-04,
       1.0282567e-05, 1.6878924e-09, 6.7422376e-04, 5.3349325e-08,
       3.4590609e-08, 2.5259002e-04, 7.1468776e-06, 2.1515353e-08,
       3.9892409e-10, 1.1916104e-05, 1.0925463e-03, 8.8620691e-05,
       1.0335469e-05, 3.7972821e-08, 3.0733030e-08, 1.0770869e-05,
       1.0666283e-08, 1.4974421e-07], dtype=float32))


In [ ]:
category_map = {
    'baby_accessories_a28': 0,
    'bags_a17': 1,
    'belts__braces_a26': 2,
    'bottoms_a3': 3,
    'dresses_a1': 4,
    'eyewear_a19': 5,
    'food_a38': 6,
    'fragrance_a31': 7,
    'gloves_mittens_a21': 8,
    'hair_accessories_a20': 9,
    'hair_care_a33': 10,
    'headwear_a18': 11,
    'home_a35': 12,
    'hosiery_a16': 13,
    'jacket_sportcoat_mg12': 14,
    'jewelry_a14': 15,
    'jewelry_care_a15': 16,
    'jumpsuits_coveralls_a4': 17,
    'makeup_a30': 18,
    'neckwear_a27': 19,
    'outerwear_a11': 20,
    'personal_care_accessories_a34': 21,
    'scarves_wraps_ponchos_a22': 22,
    'shoe_care_a13': 23,
    'shoes_a12': 24,
    'skin_body_treatment_a32': 25,
    'sleepwear_a9': 26,
    'small_leather_goods_a23': 27,
    'stationery_giftwrap_a36': 28,
    'suits_sets_wardrobers_mg5': 29,
    'swimwear_a8': 30,
    'toys_games_a37': 31,
    'tops_a2': 32,
    'underwear_lingerie_a7': 33
}